# iPAGE 
https://github.com/goodarzilab/PAGE  

In [4]:
import os

import sys
import pandas as pd 
import numpy as np
from time import time 
from glob import glob

pager_dir = '/data_gilbert/home/aarab/Projects/pager/'
# pager_dir = '/rumi/shams/abe/Projects/ipage-down/'
sys.path.append(pager_dir)

# import ipage_down as ipd

https://github.com/goodarzilab/server_wiki/tree/main/lab_softwares

In [5]:
def fdr_diff_table(file):
    df = pd.read_csv(file,sep='\t').set_index('gene_name')
    df['fdr'] = np.sign(df.log2FoldChange ) * (1 - df.pvalue)
    df[['fdr']].to_csv(file.replace('.txt','_fdr.txt'),sep='\t')

In [6]:
files = glob('diffexp/*delta_exp.txt')

for file in files:
    fdr_diff_table (file)

___
run iPAGE

In [11]:
%%bash 
export PAGEDIR=/data_gilbert/home/aarab/iPAGE

cd diffexp/
nohup ls *delta_exp*.txt \
    | parallel -j8 -k bash ~/Projects/pager/ipage_loop.sh {} \
    &> ipage.out & 
cd ../

In [13]:
cat diffexp/ipage.out

___
# `pager`
https://github.com/abearab/pager

In [9]:
def get_pvmatrix_list(parent_path,pattern):
    """
    pattern: msigdb gene set cluster name 
    """
    return glob(f'{parent_path}/*{pattern}*/pvmatrix.txt')

def get_pv_signal(exp,patterns,pdf=None):
    """
    exp: experiment name
    patterns: list of patterns
    """
    cols = pd.concat([
        ipd.pvmat2bio_signal(ipd.merge_multiple_pvmat(
            get_pvmatrix_list(f"{exp}_fdr","c1")
        ),s,n_clust=1) for s in ['up','both','down']
    ]).columns
    
    pv_signal = pd.concat([
        ipd.pvmat2bio_signal(ipd.merge_multiple_pvmat([
            pvmat for pattern in patterns
            for pvmat in 
            get_pvmatrix_list(f"{exp}",pattern) + \
            get_pvmatrix_list(f"{exp}_fdr",pattern)
        ]),s,n_clust=2) for s in ['up','both','down']
    ])
    
    pv_signal.columns = cols
    pv_signal=pv_signal[~pv_signal.duplicated()]
    
    if pdf:
        pv_signal.to_csv('temp-pvmatrix.txt',sep='\t')
    
        !bash {pager_dir}/ipage_draw_matrix.sh \
            {exp}'.txt' "temp-pvmatrix.txt" \
            {pdf} &> /dev/null
    
        !rm -v 'temp-pvmatrix.txt'
    else: 
        return pv_signal


# comb_vs_decitabine

In [51]:
pvmat_comb_vs_decitabine = get_pv_signal(
    'diffexp/comb_vs_decitabine_delta_exp',[
        'c5.go.bp',
        # 'c2.cp.kegg'
    ]
)

In [53]:
os.chdir("diffexp")
get_pv_signal('comb_vs_decitabine_delta_exp',[
    'c5.go.bp',
    'c2.cp.kegg'
],'comb_vs_decitabine_delta_exp-GO.pdf')
os.chdir("..")

removed ‘temp-pvmatrix.txt’


### BIOCARTA_SM_PATHWAY

In [158]:
ipd.detect_gs_cluster(get_pvmatrix_list("diffexp/comb_vs_decitabine_delta_exp*","c"),"BIOCARTA_SM_PATHWAY")

['diffexp/comb_vs_decitabine_delta_exp_fdr/msigdb_v7.4_c2.all/pvmatrix.txt',
 'diffexp/comb_vs_decitabine_delta_exp_fdr/msigdb_v7.4_c2.cp/pvmatrix.txt',
 'diffexp/comb_vs_decitabine_delta_exp_fdr/msigdb_v7.4_c2.cp.biocarta/pvmatrix.txt']

In [161]:
[gene for clust in ['0','1','2'] for gene in list(ipd.bin_identifier_genes(
    "diffexp/comb_vs_decitabine_delta_exp_fdr/msigdb_v7.4_c2.all",clust,'BIOCARTA_SM_PATHWAY'
).values())[0]]

['SNRNP70',
 'SNRPA1',
 'SNRPD1',
 'SNRPD3',
 'SNRPE',
 'SNRPF',
 'SRSF2',
 'U2AF1',
 'SNRPB',
 'SNRPC',
 'SNRPD2',
 'SNRPG']

### GOCC_METHYLOSOME

In [142]:
ipd.detect_gs_cluster(get_pvmatrix_list("diffexp/comb_vs_decitabine_delta_exp*","c"),"GOCC_METHYLOSOME")

['diffexp/comb_vs_decitabine_delta_exp_fdr/msigdb_v7.4_c5.all/pvmatrix.txt',
 'diffexp/comb_vs_decitabine_delta_exp_fdr/msigdb_v7.4_c5.go/pvmatrix.txt',
 'diffexp/comb_vs_decitabine_delta_exp_fdr/msigdb_v7.4_c5.go.cc/pvmatrix.txt']

In [157]:
[gene for clust in ['0','1'] for gene in list(ipd.bin_identifier_genes(
    "diffexp/comb_vs_decitabine_delta_exp_fdr/msigdb_v7.4_c5.go",clust,'GOCC_METHYLOSOME'
).values())[0]]

['CLNS1A',
 'SNRPD1',
 'SNRPD3',
 'SNRPF',
 'WDR77',
 'PRMT1',
 'PRMT5',
 'SNRPB',
 'SNRPD2',
 'SNRPE',
 'SNRPG']

## 

In [23]:
get_pv_signal('diffexp/comb_vs_decitabine_delta_exp',[
    'c5.go',
    'c2.cp.kegg'
])


,[-1.22 -1.00],[-1.00 -0.96],[-0.96 -0.79],[-0.79 -0.53],[-0.53 -0.22],[-0.22 0.08],[0.08 0.39],[0.39 0.69],[0.69 0.91],[0.91 1.00],[1.00 1.22]
GOMF_GROWTH_FACTOR_BINDING,0.246,-2.000,-1.078,-1.078,-0.771,-0.364,-0.771,0.246,-0.538,3.028,4.950
GOCC_TERTIARY_GRANULE,-1.194,-1.860,-1.499,-1.499,-0.939,1.063,-0.301,-0.413,-0.413,3.370,4.523
GOBP_T_CELL_PROLIFERATION,0.275,-2.888,-1.568,-0.328,-0.594,-0.447,-1.256,-0.447,1.436,2.554,2.862
GOCC_SPECIFIC_GRANULE,-0.338,-1.591,-0.791,-1.591,-0.791,-1.012,-0.608,0.978,0.794,2.210,2.820
GOBP_POSITIVE_REGULATION_OF_TUMOR_NECROSIS_FACTOR_SUPERFAMILY_CYTOKINE_PRODUCTION,-0.488,-1.518,-1.064,-0.488,1.156,-1.518,0.446,-0.314,-1.064,2.206,2.612
GOMF_LIPASE_ACTIVITY,0.350,-0.812,-0.812,-0.257,-0.571,-0.257,-2.066,-0.812,-0.257,2.911,2.489
GOBP_LEUKOCYTE_HOMEOSTASIS,1.099,0.389,-0.585,-0.884,-0.884,-1.891,-0.585,-0.585,-0.374,2.179,2.167
GOBP_MATURATION_OF_5_8S_RRNA,3.086,2.482,-0.430,-1.450,-0.430,-1.450,-0.430,-0.218,-0.218,-0.218,-0.800
GOBP_PROTEIN_LOCALIZATION_TO_CHROMOSOME_TELOMERIC_REGION,2.996,2.364,-0.320,-0.640,-0.640,-0.640,-1.243,0.283,-0.640,0.283,-0.643
GOBP_CLEAVAGE_INVOLVED_IN_RRNA_PROCESSING,2.679,3.372,-1.118,-0.550,-0.260,-0.550,-0.550,-0.550,-0.550,-0.260,-0.552


### GOBP_POSITIVE_REGULATION_OF_TUMOR_NECROSIS_FACTOR_SUPERFAMILY_CYTOKINE_PRODUCTION

In [25]:
ipd.detect_gs_cluster(
    get_pvmatrix_list("diffexp/comb_vs_decitabine_delta_exp","c"),
    "GOBP_POSITIVE_REGULATION_OF_TUMOR_NECROSIS_FACTOR_SUPERFAMILY_CYTOKINE_PRODUCTION"
)

['diffexp/comb_vs_decitabine_delta_exp/msigdb_v7.4_c5.all/pvmatrix.txt',
 'diffexp/comb_vs_decitabine_delta_exp/msigdb_v7.4_c5.go/pvmatrix.txt',
 'diffexp/comb_vs_decitabine_delta_exp/msigdb_v7.4_c5.go.bp/pvmatrix.txt']

In [28]:
[gene for clust in ['8','9','10'] for gene in list(ipd.bin_identifier_genes(
    "diffexp/comb_vs_decitabine_delta_exp/msigdb_v7.4_c5.all",clust,
    'GOBP_POSITIVE_REGULATION_OF_TUMOR_NECROSIS_FACTOR_SUPERFAMILY_CYTOKINE_PRODUCTION'
).values())[0]]

['AGER',
 'CLEC7A',
 'RIPK1',
 'CLU',
 'HLA-E',
 'IFIH1',
 'IL23A',
 'LBP',
 'LEP',
 'MAPKAPK2',
 'ORM1',
 'PSEN1',
 'PTAFR',
 'PTPRC',
 'TIRAP',
 'TLR1',
 'TLR4',
 'TYROBP',
 'ADAM8',
 'CCL3',
 'CD14',
 'CD86',
 'IL12B',
 'IL1A',
 'LILRA2',
 'LILRA5',
 'NOD2',
 'ORM2',
 'PTPRJ',
 'SASH3',
 'THBS1',
 'TLR9']

### GOBP_T_CELL_PROLIFERATION

In [29]:
ipd.detect_gs_cluster(
    get_pvmatrix_list("diffexp/comb_vs_decitabine_delta_exp","c"),
    "GOBP_T_CELL_PROLIFERATION"
)

['diffexp/comb_vs_decitabine_delta_exp/msigdb_v7.4_c5.all/pvmatrix.txt',
 'diffexp/comb_vs_decitabine_delta_exp/msigdb_v7.4_c5.go/pvmatrix.txt',
 'diffexp/comb_vs_decitabine_delta_exp/msigdb_v7.4_c5.go.bp/pvmatrix.txt']

In [30]:
[gene for clust in ['8','9','10'] for gene in list(ipd.bin_identifier_genes(
    "diffexp/comb_vs_decitabine_delta_exp/msigdb_v7.4_c5.all",clust,
    'GOBP_T_CELL_PROLIFERATION'
).values())[0]]

['AGER',
 'BAX',
 'BMP4',
 'CD151',
 'CD1D',
 'CD46',
 'CD81',
 'DOCK2',
 'HLA-DMB',
 'IL6ST',
 'IRF1',
 'LGALS9C',
 'PNP',
 'PRDX2',
 'PTPN6',
 'RASAL3',
 'RC3H2',
 'RIPK3',
 'TGFBR2',
 'TNFRSF1B',
 'TP53',
 'ANXA1',
 'CCDC88B',
 'CD276',
 'CD55',
 'CD70',
 'CD80',
 'CEBPB',
 'CORO1A',
 'DLG5',
 'DOCK8',
 'FKBP1B',
 'FYN',
 'HLA-A',
 'HLA-E',
 'IDO1',
 'IL23A',
 'LEP',
 'MAD1L1',
 'MARCHF7',
 'PRKAR1A',
 'PTPRC',
 'SLC11A1',
 'SOS2',
 'SPTA1',
 'TNFSF9',
 'ZBTB7B',
 'ZP3',
 'CCL5',
 'CCND3',
 'CD3E',
 'CD40LG',
 'CD86',
 'CRTAM',
 'ICOSLG',
 'IGF1',
 'IL12B',
 'IL1A',
 'IL1B',
 'IL20RB',
 'IL2RA',
 'LGALS3',
 'LILRB2',
 'PAWR',
 'PTPN22',
 'SASH3',
 'SDC4',
 'SFTPD',
 'TNFRSF14',
 'TNFRSF4',
 'TNFSF14',
 'VSIR']

### 

# comb_vs_dmso

In [8]:
pvmat_comb_vs_dmso = get_pv_signal(
    'diffexp/comb_vs_dmso_delta_exp',[
        'c5.go',
        'c2.cp.kegg'
    ]
)

In [232]:
os.chdir("diffexp")
get_pv_signal('comb_vs_dmso_delta_exp',[
    'c5.go',
    # 'h.all'
],'comb_vs_dmso_delta_exp-GO.pdf')
os.chdir("..")

removed ‘temp-pvmatrix.txt’


# Interpret!

Alex: 

> Figure 6 is something I made with Kevin
> > KRASG12 inhibition produces a driver-limited state revealing collateral dependencies https://pubmed.ncbi.nlm.nih.gov/31138768/

 
> So I think we’re trying to find out the mechanism of RG3039 + DAC synergy

### GOBP_POSITIVE_REGULATION_OF_TUMOR_NECROSIS_FACTOR_SUPERFAMILY_CYTOKINE_PRODUCTION

In [19]:
gs = 'GOBP_POSITIVE_REGULATION_OF_TUMOR_NECROSIS_FACTOR_SUPERFAMILY_CYTOKINE_PRODUCTION'

In [20]:
ipd.detect_gs_cluster(
    get_pvmatrix_list("diffexp/comb_vs_decitabine_delta_exp*","c"),gs
)

['diffexp/comb_vs_decitabine_delta_exp_fdr/msigdb_v7.4_c5.all/pvmatrix.txt',
 'diffexp/comb_vs_decitabine_delta_exp_fdr/msigdb_v7.4_c5.go/pvmatrix.txt',
 'diffexp/comb_vs_decitabine_delta_exp_fdr/msigdb_v7.4_c5.go.bp/pvmatrix.txt',
 'diffexp/comb_vs_decitabine_delta_exp/msigdb_v7.4_c5.all/pvmatrix.txt',
 'diffexp/comb_vs_decitabine_delta_exp/msigdb_v7.4_c5.go/pvmatrix.txt',
 'diffexp/comb_vs_decitabine_delta_exp/msigdb_v7.4_c5.go.bp/pvmatrix.txt']

In [44]:
[
    g for n in [10] for g in list(ipd.bin_identifier_genes(
        'diffexp/comb_vs_decitabine_delta_exp_fdr/msigdb_v7.4_c5.go.bp',str(n),gs
    ).values())[0] 
]

['ADAM8',
 'CCL3',
 'CCR2',
 'CD14',
 'CD86',
 'HLA-E',
 'IL12B',
 'IL23A',
 'LILRA2',
 'LILRA5',
 'MAPKAPK2',
 'NOD2',
 'ORM1',
 'ORM2',
 'PTAFR',
 'PTPRC',
 'PTPRJ',
 'THBS1',
 'TLR4',
 'TLR9',
 'TYROBP']

### GOBP_POSITIVE_REGULATION_OF_MYELOID_CELL_DIFFERENTIATION

> In the combo vs. DMSO, positive regulation of myeloid cell differentiation is interesting. Can we take a look at expression of genes in that annotation?

In [10]:
gs = 'GOBP_POSITIVE_REGULATION_OF_MYELOID_CELL_DIFFERENTIATION'

In [11]:
ipd.detect_gs_cluster(
    get_pvmatrix_list("diffexp/comb_vs_decitabine_delta_exp*","c"),
    "GOBP_POSITIVE_REGULATION_OF_MYELOID_CELL_DIFFERENTIATION"
)

['diffexp/comb_vs_decitabine_delta_exp_fdr/msigdb_v7.4_c5.all/pvmatrix.txt',
 'diffexp/comb_vs_decitabine_delta_exp_fdr/msigdb_v7.4_c5.go/pvmatrix.txt',
 'diffexp/comb_vs_decitabine_delta_exp_fdr/msigdb_v7.4_c5.go.bp/pvmatrix.txt']

In [12]:
pd.DataFrame([
    (n,','.join(list(ipd.bin_identifier_genes(
        'diffexp/comb_vs_decitabine_delta_exp_fdr/msigdb_v7.4_c5.go',str(n),gs
    ).values())[0])) for n in [0,1,2,8,9,10]
],columns=['clust',gs]).set_index('clust').to_dict()

{'GOBP_POSITIVE_REGULATION_OF_MYELOID_CELL_DIFFERENTIATION': {0: 'CD74,CTNNBIP1,GNAS,IL23R,PPARGC1B,PRKDC,SLC9B2,STAT5B',
  1: 'FAM210B,HMGB1,PITHD1,PRMT1,ROR2,RUNX1,SMAP1,TESC,TGFB1',
  2: 'HSPA1B,INHBA,MED1,TMEM64,ZNF16',
  8: 'ANKRD54,CSF1,FADD,HAX1,INPP5D,MAPK14,NCKAP1L,RIPK1,STAT3',
  9: 'ACVR2A,ATP6AP1,FES,HCLS1,HIF1A,HMGB2,JAG1,NOTCH2,OCSTAMP,POU4F1,RB1,RCOR1,STAT1',
  10: 'BRD1,CCR1,CD101,ETS1,EVI2B,FOS,FOXO3,GATA1,GATA2,GPR68,IL12B,IL23A,ISG15,JUN,RAB7B,RHEX,SCIN,TNF,TRIB1,TYROBP'}}

### 


> We want to look for a group of genes that show synergistic upregulation in the combination.

> For example, low expression in the DMSO condition, moderate levels of expression in DAC and/or RG3039, and then high levels of expression in the combination

> Or reverse
(downregulation)

> The comparison is a great start. But now we need to look at expression in each of the individual conditions

> I would also look at TNF genes (the first annotation in combination vs. decitabine)

In [27]:
from IPython.display import IFrame
IFrame("diffexp/comb_vs_decitabine_delta_exp-GO.pdf", width=600, height=300)

In [29]:
# pvmat_comb_vs_decitabine.index

In [30]:
gs = 'GOBP_POSITIVE_REGULATION_OF_TUMOR_NECROSIS_FACTOR_SUPERFAMILY_CYTOKINE_PRODUCTION'

In [31]:
ipd.detect_gs_cluster(
    get_pvmatrix_list("diffexp/comb_vs_decitabine_delta_exp*","c"),gs
)

['diffexp/comb_vs_decitabine_delta_exp_fdr/msigdb_v7.4_c5.all/pvmatrix.txt',
 'diffexp/comb_vs_decitabine_delta_exp_fdr/msigdb_v7.4_c5.go/pvmatrix.txt',
 'diffexp/comb_vs_decitabine_delta_exp_fdr/msigdb_v7.4_c5.go.bp/pvmatrix.txt',
 'diffexp/comb_vs_decitabine_delta_exp/msigdb_v7.4_c5.all/pvmatrix.txt',
 'diffexp/comb_vs_decitabine_delta_exp/msigdb_v7.4_c5.go/pvmatrix.txt',
 'diffexp/comb_vs_decitabine_delta_exp/msigdb_v7.4_c5.go.bp/pvmatrix.txt']

In [32]:
pd.DataFrame([
    (n,','.join(list(ipd.bin_identifier_genes(
        'diffexp/comb_vs_decitabine_delta_exp_fdr/msigdb_v7.4_c5.go',str(n),gs
    ).values())[0])) for n in [0,1,2,8,9,10]
],columns=['clust',gs]).set_index('clust').to_dict()

{'GOBP_POSITIVE_REGULATION_OF_TUMOR_NECROSIS_FACTOR_SUPERFAMILY_CYTOKINE_PRODUCTION': {0: 'AZU1,DDT,LGALS9,LPL,MMP8',
  1: 'BCL10,DHX9,HDAC2,HMGB1',
  2: 'ARHGEF2,MAVS,MIF,NOD1',
  8: 'CLU,FADD,JAK2,RIPK1,STAT3',
  9: 'IFIH1,IL1A,PSEN1,SASH3,SPN',
  10: 'ADAM8,CCL3,CCR2,CD14,CD86,HLA-E,IL12B,IL23A,LILRA2,LILRA5,MAPKAPK2,NOD2,ORM1,ORM2,PTAFR,PTPRC,PTPRJ,THBS1,TLR4,TLR9,TYROBP'}}

___
# 

In [162]:
!date

Thu Aug  4 18:13:12 PDT 2022
